In [33]:


import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import os
import gmplot
import torch
from torch import nn
from tqdm import tqdm
from tqdm import trange
import time
from tqdm import trange
import h3
from collections import OrderedDict
from sklearn.preprocessing import  OneHotEncoder
import move_fromd2l as d2l

def geo_t_h3(data):
    h3_list =OrderedDict()
    for i in data:
        a = h3.geo_to_h3(i[0], i[1], 10)
        # print(a)
        h3_list.setdefault(a)
    #这这里去掉h3的重复
    return h3_list
def h3_t_geo(data):
    new_list = []
    for i in data:
        i =h3.h3_to_geo(i)
        new_list.append(i)
    return new_list

# draw(,130,10)
# list,文件名，放大率
def draw(list,number,b):
    new_list = torch.tensor(list)
    lat = []
    lng = []
    for i in new_list:
        lat.append(i[0])
        lng.append(i[1])
    lat = torch.stack(lat)
    lng =torch.stack(lng)
    gmap = gmplot.GoogleMapPlotter(lat[0], lng[0], b)
    gmap.plot(lat, lng,color='r',lw=10)  #描绘轨迹点
    gmap.draw("user{}.html".format(number))   #显示图
    print("over")
# 这个提取出来有5个维度
def dataset(user,start,end,step):
    # user:第几个用户
    # filenumber:取前面几天数据
    # step：隔了多少步取一次
    userdata =  '../Geolife Trajectories 1.3/Data/'+user+'/Trajectory/'
    filelist = os.listdir(userdata)  #返回指定路径下所有文件和文件夹的名字，并存放于一个列表中
    filelist.sort()
    names = ['lat','lng','zero','alt','days','date','time']
    df_list = [# f为文件索引号，header为列数，names为列表列名，index_col为行索引的列编号或列名
    pd.read_csv(userdata + f,header=6,names=names,index_col=False)
    for f in filelist[start:end]]
    df = pd.concat(df_list, ignore_index=True) #表格列字段不同的表合并
    df.drop(['zero', 'days'], axis=1, inplace=True) #drop函数默认删除行，列需要加axis = 1
    df_min = df.iloc[::step, :]
    return df_min
# 这个提取出来有2个维度


def synthetic_data(df_min):
    a =df_min['lat'].tolist()
    b = df_min['lng'].tolist()
    a = torch.tensor(a,dtype=torch.float,requires_grad=True).reshape((-1, 1))
    b = torch.tensor(b,dtype=torch.float,requires_grad=True).reshape((-1, 1))
    features = torch.concat([a,b],1)
    return features
# 将(lat,lng)变成哈希值
def generate_h3_list(data):
    alist = geo_t_h3(data)
    print(type(alist))
    LIST = list(alist.keys())
    return np.array(LIST)




# 这个all_data_h3_list可以先shuffle一下






In [34]:
# 这个提取出来有5个维度
train_dataset = dataset("006",0,20,60)
test_dataset =  dataset("006",20,25,60)

# 这个提取出来有2个维度
train_data = synthetic_data(train_dataset)
test_data = synthetic_data(test_dataset)
# concat()
all_data =torch.concat([train_data,test_data],0)



Train_h3_list  = generate_h3_list(train_data)
TESETLIST  = generate_h3_list(test_data)
vocab = generate_h3_list(all_data)
print(len(Train_h3_list))
print(len(vocab))

encoder = OneHotEncoder(sparse=False).fit(vocab.reshape(-1,1))
def encoding(data):
    return encoder.transform(data.reshape(-1,1))
def decoding(one_hot_data):
    return encoder.inverse_transform(one_hot_data)




# 可以通过idnex找到对应的h3
index_h  = dict(enumerate(vocab))
# 可以通过h3编码找到对应的index
h_index ={h3:i for i ,h3 in index_h.items()}




def label_encode(data):
    return np.array([h_index[ch] for ch in data])

def label_decode(data):
    return np.array([index_h[ch] for ch in data])


#y是一个列表，k是步长
def dataloader(y,k):
    data =[]
    for i in range(len(y)-k+1):
        indata = y[i:i+k]
        outdata = y[i+k:i+k+1]
        data.append((indata,outdata))
    return data

train_dataloader = dataloader(Train_h3_list,10)
test_dataloader =dataloader(test_data,10)
# for i,j in train_dataloader:
#     print(i,j)


<class 'collections.OrderedDict'>
<class 'collections.OrderedDict'>
<class 'collections.OrderedDict'>
278
366


In [35]:
class RNN(nn.Module):
    def __init__(self,vocab) -> None:
        super().__init__()
        self.vocab = vocab
        self.model =nn.LSTM(
            input_size=vocab,
            hidden_size=320,
            batch_first=True,
            bias=True
        )

        self.output = nn.Sequential(
            nn.Linear(320,vocab),
            nn.ReLU())

    def forward(self,x):
        r_out,(h_out,c_out) = self.model(x.view(1,10,self.vocab) ,None)
        outdata = self.output(r_out[:,-1,:])
        return outdata
net = RNN(len(vocab)).to(d2l.try_gpu())
optimizer = torch.optim.SGD(net.parameters(),lr = 0.1,momentum=0.8)
optimizer_scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=20, gamma=0.5)
loss_function = nn.CrossEntropyLoss(reduction='sum')
#这里无论是sum还是mean的结果是一样的




In [36]:
for epoch in trange(1000):
    loss_ =0
    optimizer_scheduler.step()
    optimizer.step()
    for i , k in train_dataloader[:-1]:
        optimizer.zero_grad()
        a = encoder.transform(i.reshape(-1,1))
        a =torch.tensor(a).to(torch.float32).to(d2l.try_gpu())
        k = label_encode(k)
        k =torch.tensor(k).to(torch.long).to(d2l.try_gpu())
        loss = loss_function(net(a),k)
        print(loss)
        loss.backward()
        optimizer.step()
        loss_ = loss_+loss
    print(loss_)

  0%|          | 0/1000 [00:00<?, ?it/s]

tensor(5.8506, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(5.9089, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(5.8864, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(5.9183, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(5.9139, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(5.9198, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(5.9008, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(5.9207, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(5.8827, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(5.9219, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(5.9225, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(5.8995, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(5.8938, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(5.9243, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(5.9251, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(5.9112, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(5.9264, device='cuda:0', grad_fn=

C:\Users\zhangliyu\.conda\envs\limuconda\lib\site-packages\torch\optim\lr_scheduler.py:129: UserWarning: Detected call of `lr_scheduler.step()` before `optimizer.step()`. In PyTorch 1.1.0 and later, you should call them in the opposite order: `optimizer.step()` before `lr_scheduler.step()`.  Failure to do this will result in PyTorch skipping the first value of the learning rate schedule. See more details at https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate
  warnings.warn("Detected call of `lr_scheduler.step()` before `optimizer.step()`. "


tensor(5.9270, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(5.9206, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(5.9298, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(5.9308, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(5.9315, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(5.9320, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(5.9323, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(5.9314, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(5.9328, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(5.9291, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(5.9348, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(5.9357, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(5.9361, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(5.9367, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(5.9372, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(5.9373, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(5.9371, device='cuda:0', grad_fn=

  0%|          | 1/1000 [00:01<25:51,  1.55s/it]

...

  3%|▎         | 30/1000 [00:47<26:17,  1.63s/it]

tensor(5.9150, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(5.9154, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(5.9132, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(5.9153, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(5.9190, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(5.9130, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(5.9292, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(6.1084, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.0785, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.0857, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(5.9413, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(5.9524, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(1371.2087, device='cuda:0', grad_fn=<AddBackward0>)
tensor(0.0288, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.0381, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.1066, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(6.0497, device='cuda:0', grad_fn=<

  3%|▎         | 30/1000 [00:49<26:40,  1.65s/it]

tensor(5.9329, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(5.9198, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(5.9221, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(5.9315, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(5.9409, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(5.9372, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(5.9352, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(5.9334, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(5.9326, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(5.9269, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(5.9254, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(5.9238, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(5.9223, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(5.9194, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(5.9176, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(5.9160, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(5.9152, device='cuda:0', grad_fn=

KeyboardInterrupt: 